In [ ]:
import cv2  # Thư viện xử lý ảnh OpenCV
import os   # Thư viện thao tác với hệ thống tệp
import shutil  # Thư viện hỗ trợ sao chép tệp

# Đường dẫn đến thư mục chứa ảnh đầu vào
input_folder = 'D:/projectMV/mask/FMD_DATASET/incorrect_mask/mc'  # Thay đổi nếu cần

# Thư mục để lưu ảnh sau khi cắt khuôn mặt
output_folder = 'D:/projectMV/mask/FMD_DATASET/incorrect_mask/crop'  # Thay đổi nếu cần

# Tải bộ phân loại Haar Cascade để phát hiện khuôn mặt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Duyệt qua tất cả các tệp trong thư mục ảnh đầu vào
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Chỉ xử lý các tệp ảnh
        # Tạo đường dẫn đầy đủ đến ảnh
        image_path = os.path.join(input_folder, filename)

        # Đọc ảnh gốc
        img = cv2.imread(image_path)

        # Chuyển ảnh sang thang độ xám để phát hiện khuôn mặt
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Dùng cascade để phát hiện khuôn mặt trong ảnh xám
        faces = face_cascade.detectMultiScale(
            gray_image, 
            scaleFactor=1.1, 
            minNeighbors=3, 
            minSize=(40, 40)
        )

        # Nếu phát hiện ít nhất 1 khuôn mặt
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Cắt khuôn mặt từ ảnh gốc
                face_cropped = img[y:y + h, x:x + w]

                # Tạo đường dẫn lưu ảnh khuôn mặt đã cắt
                output_path = os.path.join(output_folder, filename)

                # Lưu ảnh đã cắt vào thư mục output
                cv2.imwrite(output_path, face_cropped)

                # print(f"Đã cắt và lưu khuôn mặt từ {filename} tại: {output_path}")
        else:
            # Nếu không phát hiện khuôn mặt, sao chép ảnh gốc sang thư mục output
            source_file = image_path
            destination_file = os.path.join(output_folder, filename)
            shutil.copy2(source_file, destination_file)
            # print(f"Không phát hiện khuôn mặt nào trong {filename}, đã copy ảnh gốc.")

# In thông báo hoàn thành
print('Hoàn thành xử lý tất cả ảnh trong thư mục.')

In [ ]:
import cv2  # Thư viện xử lý ảnh OpenCV
import os   # Thư viện thao tác với hệ thống tệp
import shutil  # Hỗ trợ sao chép hoặc di chuyển tệp

# Hàm để cắt và lưu các vùng chứa mắt phát hiện được
def cut_and_save_eyes(img, eyes, save_path, base_filename):
    count = 0
    for (x, y, w, h) in eyes:
        # Cắt vùng mắt từ ảnh gốc
        eye_img = img[y:y+h, x:x+w]

        # Đặt tên file lưu theo thứ tự mắt phát hiện
        eye_filename = f"{base_filename}_eye_{count}.jpg"

        # Lưu ảnh mắt đã cắt
        cv2.imwrite(os.path.join(save_path, eye_filename), eye_img)
        count += 1

# Thư mục chứa ảnh đầu vào (ảnh khuôn mặt đã được crop trước đó)
input_folder = "D:/projectMV/mask/FMD_DATASET/incorrect_mask/crop"

# Thư mục lưu ảnh **không có mắt** (tức là không phát hiện được mắt)
output_no_eyes = "D:/projectMV/mask/FMD_DATASET/incorrect_mask/final"

# Tạo thư mục lưu ảnh nếu chưa tồn tại
os.makedirs(output_no_eyes, exist_ok=True)
# Nếu cần lưu riêng ảnh mắt, bạn có thể mở lại dòng này:
# os.makedirs(output_eyes, exist_ok=True)

# Tải bộ Haar Cascade dùng để phát hiện mắt
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Duyệt qua tất cả ảnh trong thư mục đầu vào
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):  # Kiểm tra phần mở rộng ảnh
        img_path = os.path.join(input_folder, filename)

        # Đọc ảnh và chuyển sang ảnh xám
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Phát hiện mắt trong ảnh
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        # Lấy tên gốc của file (không bao gồm phần mở rộng)
        base_name = os.path.splitext(filename)[0]

        if len(eyes) > 0:
            # Nếu phát hiện được mắt
            # Gọi hàm để cắt và lưu từng vùng mắt (hiện tại đang comment)
            # cut_and_save_eyes(img, eyes, output_eyes, base_name)
            print("1")  # Thông báo phát hiện mắt
        else:
            # Nếu không phát hiện được mắt, di chuyển ảnh sang thư mục "final"
            shutil.move(img_path, output_no_eyes)
            print("2")  # Thông báo không phát hiện mắt
